In [4]:
# 必要ライブラリのインストール
from PIL import Image
import numpy as np

In [5]:
def MSE(x, y):
    l = len(x)
    sum = 0.0
    for i in range(l):
        diff = float(x[i]) - float(y[i])
        sum += (diff * diff)
    return sum / l

In [6]:
def phi(k, N):
	if k == 0:
		return np.ones(N) / np.sqrt(N)
	else:
		return np.sqrt(2.0 / N) * np.cos((k * np.pi / (2 * N)) * (np.arange(N) * 2 + 1))

def DCT_mat_reducedto1d(N):
	phi_1d = np.array([phi(k, N) for k in range(N)])
	phi_2d = np.zeros((N,N,N,N))
	
	for i in range(N):
		for j in range(N):
			phi_i, phi_j = np.meshgrid(phi_1d[i], phi_1d[j])
			phi_2d[i, j] = phi_i*phi_j
			
	return phi_2d.reshape(N * N, N * N)

def DCT_mat_jpg():
	DCT_mat = DCT_mat_reducedto1d(8)
	res = []
	for i in range(8):
		for j in range(8):
			cond = (i == 0) or (j == 0) or (i + j <= 4)
			if cond: res.append(DCT_mat[i * 8 + j])
	return np.array(res)

In [7]:
def DCT_mat_jpg2():
	res = DCT_mat_reducedto1d(8)
	for i in range(8):
		for j in range(8):
			cond = (i == 0) or (j == 0) or (i + j <= 4)
			if not cond:
				for k in range(64): res[i * 8 + j, k] = 0.0
	return res

In [48]:
C0 = DCT_mat_reducedto1d(8)
C1 = DCT_mat_jpg2()
D = C0.T.dot(C1)
D2 = D * D
n = np.ones(64)
E = n.dot(D2)
sum = np.sum(np.sqrt(E))
n_all = 1.0 * 64
n_list = n_all * np.sqrt(E) / sum
n_mat = n_list.reshape(8, 8)

In [49]:
coef_list = [[[] for x in range(15)] for y in range(15)]
for i in range(64):
    for j in range(64):
        x1 = i % 8
        y1 = i // 8
        x2 = j % 8
        y2 = j // 8
        coef_list[x1 - x2 + 7][y1 - y2 + 7].append(D[i, j] * 64)

In [50]:
ave_mat = [[0.0 for i in range(15)]for j in range(15)]
std_mat = [[0.0 for i in range(15)]for j in range(15)]
for i in range(15):
    for j in range(15):
        ave_mat[i][j] = np.average(coef_list[i][j])
        std_mat[i][j] = np.std(coef_list[i][j])

In [51]:
coef_list[7][7]

[33.504294321200284,
 23.142255971524566,
 22.859825123396952,
 22.76776695296637,
 22.767766952966365,
 22.85982512339695,
 23.14225597152457,
 33.50429432120028,
 23.14225597152457,
 18.018058132495753,
 19.23223304703363,
 18.847281657789594,
 18.84728165778959,
 19.232233047033628,
 18.018058132495753,
 23.142255971524563,
 22.859825123396952,
 19.232233047033628,
 18.203767274452222,
 17.150637247288884,
 17.150637247288884,
 18.203767274452215,
 19.232233047033628,
 22.859825123396945,
 22.767766952966365,
 18.84728165778959,
 17.150637247288884,
 18.273880271851738,
 18.273880271851738,
 17.150637247288884,
 18.84728165778959,
 22.767766952966358,
 22.767766952966365,
 18.84728165778959,
 17.150637247288884,
 18.27388027185174,
 18.273880271851738,
 17.150637247288884,
 18.84728165778959,
 22.76776695296636,
 22.859825123396945,
 19.232233047033624,
 18.203767274452215,
 17.150637247288884,
 17.15063724728888,
 18.20376727445221,
 19.232233047033624,
 22.85982512339694,
 23.1422

In [53]:
for i in range(15):
    for j in range(15):
        print(round(ave_mat[i][j]), end = '\t')
    print('')

4	1	-1	0	1	0	-3	3	-3	0	1	0	-1	1	4	
1	-1	-2	0	1	0	-2	4	-2	0	1	0	-2	-1	1	
-1	-2	-1	1	1	0	-3	4	-3	0	1	1	-1	-2	-1	
0	0	1	1	1	-1	-3	3	-3	-1	1	1	1	0	0	
1	1	1	1	0	-1	-3	4	-3	-1	0	1	1	1	1	
0	0	0	-1	-1	-2	-1	7	-1	-2	-1	-1	0	0	0	
-3	-2	-3	-3	-3	-1	1	11	1	-1	-3	-3	-3	-2	-3	
3	4	4	3	4	7	11	21	11	7	4	3	4	4	3	
-3	-2	-3	-3	-3	-1	1	11	1	-1	-3	-3	-3	-2	-3	
0	0	0	-1	-1	-2	-1	7	-1	-2	-1	-1	0	0	0	
1	1	1	1	0	-1	-3	4	-3	-1	0	1	1	1	1	
0	0	1	1	1	-1	-3	3	-3	-1	1	1	1	0	0	
-1	-2	-1	1	1	0	-3	4	-3	0	1	1	-1	-2	-1	
1	-1	-2	0	1	0	-2	4	-2	0	1	0	-2	-1	1	
4	1	-1	0	1	0	-3	3	-3	0	1	0	-1	1	4	


In [54]:
for i in range(15):
    for j in range(15):
        print(round(std_mat[i][j]), end = '\t')
    print('')

0	0	2	2	1	1	1	2	1	1	1	2	2	0	0	
0	0	1	1	1	1	1	1	1	1	1	1	1	0	0	
2	1	1	0	1	1	1	2	1	1	1	0	1	1	2	
2	1	0	1	1	1	1	2	1	1	1	1	0	1	2	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
2	1	2	2	2	1	2	4	2	1	2	2	2	1	2	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
2	1	0	1	1	1	1	2	1	1	1	1	0	1	2	
2	1	1	0	1	1	1	2	1	1	1	0	1	1	2	
0	0	1	1	1	1	1	1	1	1	1	1	1	0	0	
0	0	2	2	1	1	1	2	1	1	1	2	2	0	0	


In [58]:
import copy

D_prime = D
for i in range(64):
    for j in range(64):
        if i != j: D_prime[i, j] += (D[i, i] / 63.0)
        else: D_prime[i, j] = 0.0

coef_list2 = [[[] for x in range(15)] for y in range(15)]
for i in range(64):
    for j in range(64):
        x1 = i % 8
        y1 = i // 8
        x2 = j % 8
        y2 = j // 8
        coef_list2[x1 - x2 + 7][y1 - y2 + 7].append(D_prime[i, j] * 64)

ave_mat2 = [[0.0 for i in range(15)]for j in range(15)]
std_mat2 = [[0.0 for i in range(15)]for j in range(15)]
for i in range(15):
    for j in range(15):
        ave_mat2[i][j] = np.average(coef_list2[i][j])
        std_mat2[i][j] = np.std(coef_list2[i][j])

for i in range(15):
    for j in range(15):
        print(round(ave_mat2[i][j]), end = '\t')
    print('')

4	1	-1	0	1	0	-3	3	-2	0	1	0	-1	1	5	
1	-1	-2	0	1	0	-2	4	-2	1	2	0	-1	-1	1	
-1	-2	-1	1	1	0	-3	4	-2	0	2	1	-1	-1	-1	
0	0	1	1	1	-1	-3	3	-3	0	2	2	1	0	0	
1	1	1	1	0	-1	-3	4	-3	-1	1	2	2	2	1	
0	0	0	-1	-1	-2	-1	7	-1	-1	-1	0	0	1	0	
-3	-2	-3	-3	-3	-1	1	11	1	-1	-3	-3	-2	-2	-2	
3	4	4	3	4	7	11	0	11	7	4	4	4	4	4	
-3	-2	-3	-3	-3	-1	1	11	1	-1	-3	-3	-2	-2	-2	
0	0	0	-1	-1	-2	-1	7	-1	-1	-1	0	0	1	0	
1	1	1	1	0	-1	-3	4	-3	-1	1	2	2	2	1	
0	0	1	1	1	-1	-3	4	-3	0	2	2	1	0	0	
-1	-2	-1	1	1	0	-3	4	-2	0	2	1	-1	-1	-1	
1	-1	-2	0	1	0	-2	4	-2	1	2	0	-1	-1	1	
4	1	-1	0	1	0	-3	4	-2	0	1	0	-1	1	5	


In [59]:
for i in range(15):
    for j in range(15):
        print(round(std_mat2[i][j]), end = '\t')
    print('')

0	0	2	2	1	1	1	2	1	1	1	1	2	0	0	
0	0	1	1	1	1	1	1	1	1	1	1	1	0	0	
2	1	1	0	1	1	1	2	1	1	1	1	1	1	2	
2	1	0	1	1	1	1	2	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
2	1	2	2	2	1	2	0	2	1	2	2	1	1	2	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	1	1	1	1	1	1	1	1	
1	1	1	1	1	1	1	2	1	1	1	1	1	1	1	
2	1	0	1	1	1	1	2	1	1	1	1	1	1	1	
2	1	1	0	1	1	1	1	1	1	1	1	1	1	2	
0	0	1	1	1	1	1	1	1	1	1	1	1	0	0	
0	0	2	2	1	1	1	2	1	1	1	1	2	0	0	
